#### AnimateLCM-svd-xt

In [ ]:
import torch
from pathlib import Path
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
from diffusers.models.attention_processor import AttnProcessor
from safetensors import safe_open
import gc
import requests
import os

# Set the specific GPU you want to use (e.g., GPU 2)
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Change "0" to the index of the desired GPU

# Set the device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Example usage
# model.to(device)  # Move your model to the specified device

lcm_scheduler_url = "https://huggingface.co/spaces/wangfuyun/AnimateLCM-SVD/raw/main/lcm_scheduler.py"

if not os.path.exists('lcm_scheduler.py'):
    r = requests.get(lcm_scheduler_url)
    with open("lcm_scheduler.py", "w") as f:
        f.write(r.text)

from lcm_scheduler import AnimateLCMSVDStochasticIterativeScheduler

model = 'svd-xt'
if model == 'svd-xt':
    pretrained_model_path = '/home/yiming/pretrained_models/stable-video-diffusion-img2vid-xt'
    online_model_path = 'stabilityai/stable-video-diffusion-img2vid-xt'
    model_path = '/home/yiming/pretrained_models/AnimateLCM/AnimateLCM_SVD/AnimateLCM-SVD-xt.safetensors'
elif model == 'svd-xt-1-1':
    pretrained_model_path = '/home/yiming/pretrained_models/stable-video-diffusion-img2vid-xt-1-1'
    online_model_path = 'stabilityai/stable-video-diffusion-img2vid-xt-1-1'
    model_path = '/home/yiming/pretrained_models/AnimateLCM/AnimateLCM_SVD/AnimateLCM-SVD-xt-1.1.safetensors'
else:
    raise ValueError(f"Model {model} not found")

noise_scheduler = AnimateLCMSVDStochasticIterativeScheduler(
    num_train_timesteps=40,
    sigma_min=0.002,
    sigma_max=700.0,
    sigma_data=1.0,
    s_noise=1.0,
    rho=7,
    clip_denoised=False,
)
if os.path.exists(pretrained_model_path):
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        pretrained_model_path,
        variant="fp16",
        scheduler=noise_scheduler,
    ).to(device)
else:
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        online_model_path,
        variant="fp16",
        scheduler=noise_scheduler,
    ).to(device)

# pipe.unet.set_attn_processor(AttnProcessor())
state_dict = {}
LCM_LORA_PATH = Path(
    "/home/yiming/pretrained_models/AnimateLCM/AnimateLCM_SVD/AnimateLCM-SVD-xt.safetensors",
)
with safe_open(LCM_LORA_PATH, framework="pt", device="cpu") as f:
    for key in f.keys():
        state_dict[key] = f.get_tensor(key)
missing, unexpected = pipe.unet.load_state_dict(state_dict, strict=True)

unet = pipe.unet
unet.eval()
vae = pipe.vae
vae.eval()
image_encoder = pipe.image_encoder
image_encoder.eval()

# Load the conditioning image
img_dir = '/home/yiming/project/MyProjects/VisionGen/assets/images'
img_names = os.listdir(img_dir)
img_names = [os.path.join(img_dir, img_name) for img_name in img_names]

for img_name in img_names:
    image = load_image(img_name)
    image = image.resize((1024, 576))
    frames = pipe(
        image,
        num_inference_steps=4,
        motion_bucket_id=127,
        num_frames=14,
        height=576,
        width=1024,
        generator=torch.manual_seed(12342)
    ).frames[0]

del pipe
gc.collect()